In [2]:
####

# Code generated for "Bill of Lading" dataset registered under "Bill of Lading (PIERS)" package

# Documentation and Installations Instructions link: https://catalogue.datalake.ihsmarkit.com (please follow the links to "Documentation")

# This code is compatible with latest version of the Data Lake command line interface hosted on pypi.org: https://pypi.org/project/dli/

# To run with python interpreter (preferably using 3.x version)

####



# Import Libraries

import dli
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib
from sklearn.linear_model import LinearRegression
import os, glob
from os import listdir
import gc
from datetime import datetime



print('#All Libaries are imported already#')

#All Libaries are imported already#


In [2]:
# From cleaned Bill of Lading data import a set of company ticker for mapping later on
BoL_ticker = 'matched_fcompany_ihsm_ticker'
Tickers = pd.read_excel(rf'C:\Users\Thomas TH Chow\Desktop\Datalake\Tickers\IHS {BoL_ticker} Common List.xlsx')['Common Ticker'].to_list()
print(len(Tickers))

808


In [5]:
# Check the trade volume of a company with a country and convert it into a matrix
def Max_conc(df, column, value_column):
    df[value_column] = df[value_column].apply(pd.to_numeric)
    df[column].replace({np.nan:'Unknown'},inplace= True)
    Countrytable = pd.pivot_table(df, values = value_column, index=['vyear', 'vmonth'],
                    columns=[column], aggfunc=np.sum, fill_value = 0)
    Countrytable['Total'] = Countrytable.sum(axis=1)
    Countrytable.iloc [:, 0:] = Countrytable.iloc [:, 0:].apply (lambda x: x/Countrytable ['Total'])
    Countrytable = Countrytable.drop(columns = ["Total"])
    Countrytable[f'Max_{value_column}_{column}'] = Countrytable.max(axis=1)
    return Countrytable[f'Max_{value_column}_{column}']

# Take log to the columns
def log(df, column):
    index = df.columns.get_loc("harm4")    
    df.insert(index, f'{column}_log10', np.log10(df[column] + 1))
    return df
# Check the number of times that a company traded with a country and convert it into a matrix
def Count_country(df, column):
    df[column].replace({np.nan:'Unknown'},inplace= True)
    Countrytable = pd.pivot_table(df, values='estimated_dollarvalue', index=['vyear', 'vmonth'],
                    columns=[column], aggfunc=pd.Series.nunique, fill_value = 0)
    return Countrytable
# Count the number of days since last shippment occurs
def Count_zero(df, column):
    index = df.columns.get_loc(column)
    df.insert(index + 1, f'days_since_last_{column}', np.nan)
    Count = 0
    j = 0
    firsttrade = False
    try:
        while firsttrade == False:
            if df.loc[df.index[j], column] == 0:
                j += 1
                continue
            else:
                firsttrade = True
    except:
        pass
    
    for i in range(j,len(df)):
        cell_value = df.loc[df.index[i], column]
        if (cell_value == 0):
            Count += 1
        else:
            Count = 0
        df.loc[df.index[i], f'days_since_last_{column}'] = Count
    return df
# Compute the rolling std of a column data
def Rolling_vol(df, column, month):
    index = df.columns.get_loc('harm4')
    df.insert(index , f'{column}_{month}Mvol', df.rolling(window = month)[column].std())
    return df
# Compute the rolling max of a column data
def Rolling_max(df, column, month):
    index = df.columns.get_loc('harm4')
    df.insert(index , f'{column}_{month}MMax', df.rolling(window = month)[column].max())
    return df

# Compute the rolling mean of a column data
def Moving_average(df, column, month):
    index = df.columns.get_loc('harm4')
    df.insert(index , f'{column}_SMA{month}', df.rolling(window = month)[column].mean())
    return df

#Test code and supplement function for understanding the moving average function, they are equivalent
def Moving_average_supplement(df, column, month):
    index = df.columns.get_loc('harm4')
    df.insert(index, f'{column}_SMA{month}', np.nan)
    for i in range(0,len(df)-month):
        Sum = 0
        for j in range(0,month):
            Sum += df.iloc[i+j,index]
        df.loc[df.index[i+month-1], f'{column}_SMA{month}'] = np.round((Sum/month),1)
    return df

def Oscillator(df, column_short, column_long):
    index = df.columns.get_loc(column_long)
    df.insert(index + 1, f'Osci_{column_short}_{column_long}', (df[column_short] - df[column_long]))
    return df

#Two important functions to Filter and Organise the columns in CDS and BoL datasets
#Function for transforming CDS datasets by getting Month, year and PD
def Transform_CDS(COMPANY_CDS, Com_Ticker):
    years = [2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    COMPANY_CDS = COMPANY_CDS[(COMPANY_CDS['Tenor'] == 1) & (COMPANY_CDS['PrimaryCoupon']== True) & (COMPANY_CDS['PrimaryCurve']== True)]
    COMPANY_CDS = COMPANY_CDS[['PD','Year','Month','Ticker']]
    try:
        # Create new column to indicate the default status of the company
        COMPANY_CDS['Default'] = np.where(COMPANY_CDS['PD'].isnull(), True, False)
        COMPANY_CDS = COMPANY_CDS.reset_index(drop = True)
        # Remove CDS data after the company defaulted
        COMPANY_CDS = COMPANY_CDS.loc[: COMPANY_CDS[(COMPANY_CDS['Default'] == True)].index[0], :]
    except:
        pass
    COMPANY_CDS = COMPANY_CDS.groupby(['Year','Month'],dropna = False).mean()
    COMPANY_CDS = COMPANY_CDS.reindex(pd.MultiIndex.from_product([years, months], names=['Year', 'Month']))
    COMPANY_CDS['Ticker'] = Com_Ticker
    return COMPANY_CDS
#Function for transforming bill of lading datasets
def Transform_BoL(Company_BoL):
    years = [2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    TradingCompany = Max_conc(Company_BoL, 'us_company_name', 'qty')
    TradingCompany1 = Max_conc(Company_BoL, 'us_company_name', 'estimated_dollarvalue')
    #Group all numerical values and sum them up
    Numericals = Company_BoL[['vyear','vmonth','qty', 'kilos', 'metric_tons', 'estimated_dollarvalue']].apply(pd.to_numeric).groupby(['vyear','vmonth']).sum()
    #Group all unique items by counting their unique values
    Unique_items = Company_BoL[['vyear','vmonth','harm4']].groupby(['vyear','vmonth']).nunique()
    Company_BoL = pd.concat([Numericals, Unique_items, TradingCompany,TradingCompany1], axis=1)
    Company_BoL = Company_BoL.sort_index(ascending=True)
    Company_BoL["Unit_price_qty"] = Company_BoL["estimated_dollarvalue"]/Company_BoL["qty"]
    Company_BoL["Unit_price_kilos"] = Company_BoL["estimated_dollarvalue"]/Company_BoL["kilos"]
    Company_BoL = Company_BoL.reindex(pd.MultiIndex.from_product([years, months], names=['Year', 'Month']),fill_value=0)
    #Twisting columns into X model input
    Company_BoL = Count_zero(Company_BoL, 'qty')
    #Unit Price Engineering
    Company_BoL = Moving_average(Company_BoL, "Unit_price_qty", 12)
    Company_BoL = Moving_average(Company_BoL, "Unit_price_qty", 36)
    Company_BoL = log(Company_BoL, 'Unit_price_qty_SMA12')
    Company_BoL = log(Company_BoL, 'Unit_price_qty_SMA36')
    Company_BoL = Oscillator(Company_BoL, 'Unit_price_qty_SMA12_log10', 'Unit_price_qty_SMA36_log10')
    #Trading Volume Engineering
    Company_BoL = Moving_average(Company_BoL, 'estimated_dollarvalue', 12)
    Company_BoL = Moving_average(Company_BoL, 'estimated_dollarvalue', 36)
    Company_BoL = Oscillator(Company_BoL, 'estimated_dollarvalue_SMA12', 'estimated_dollarvalue_SMA36')
    Company_BoL = log(Company_BoL, 'estimated_dollarvalue_SMA12')
    Company_BoL = log(Company_BoL, 'estimated_dollarvalue_SMA36')
    Company_BoL = Oscillator(Company_BoL, 'estimated_dollarvalue_SMA12_log10', 'estimated_dollarvalue_SMA36_log10')
    Company_BoL = Rolling_vol(Company_BoL, 'estimated_dollarvalue', 12)
    Column_drop = ['qty', 'kilos', 'metric_tons', 'estimated_dollarvalue',"Unit_price_qty","Unit_price_kilos"]
    Company_BoL = Company_BoL.drop(columns = Column_drop)
    return Company_BoL


In [4]:
#Sort and Filter all companies and output two company level Excel file each for cleansing and organising
Error_tickers = []
for Ticker in Tickers:
    try:
        print(rf'[{datetime.now()}]Converting {Ticker}...')
        input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\CDS"
        cds_files = [file for file in listdir(input_path) if file.endswith('.parquet.gzip')]
        cds_dict = {}
        for i in range (0,len(cds_files)):
            # read and find company level data in processed raw CDS datasets
            cds_file = cds_files[i]
            path = os.path.join(input_path, cds_file)
            temp_df = pd.read_parquet(path, columns = ['Tenor','PrimaryCoupon','PrimaryCurve','PD','Year','Month','Ticker'])
            # filter line
            temp_df = temp_df[temp_df['Ticker'] == Ticker]
            # Use dictionaries to reduce time complexity and incl all company level data
            temp_dict={}
            if len(temp_df) !=0:
                temp_dict = temp_df.to_dict('index')
                cds_dict.update(temp_dict)
            index = temp_df.index
            del temp_df
            del temp_dict
            gc.collect()
        Company_CDS = pd.DataFrame.from_dict(cds_dict, orient='index')
        # Features engineering and transformation
        Company_CDS = Transform_CDS(Company_CDS, Ticker)

        input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading"
        Bill_Temps = [file for file in listdir(input_path) if file.endswith('.parquet.gzip')]
        Bill_dict = {}
        for j in range (0,len(Bill_Temps)):
            # read and find company level data in processed raw BoL datasets
            Bill_Temp = Bill_Temps[j]
            path = os.path.join(input_path, Bill_Temp)
            Temp_df = pd.read_parquet(path,columns = ['harm4', 'vyear','vmonth','qty', 'kilos', 'metric_tons', 'estimated_dollarvalue','us_company_name','foreign_company_country','matched_fcompany_ihsm_ticker'])
            # filter line
            Temp_df = Temp_df[Temp_df['matched_fcompany_ihsm_ticker'] == Ticker]
            # Use dictionaries to reduce time complexity and incl all company level data
            Temp_dict={}
            if len(Temp_df) !=0:
                Temp_dict = Temp_df.to_dict('index')
                Bill_dict.update(Temp_dict)
            del Temp_df
            del Temp_dict
            gc.collect()
        Company_bill_df = pd.DataFrame.from_dict(Bill_dict, orient='index')
        # Features engineering and transformation
        Company_bill_df = Transform_BoL(Company_bill_df)
        Company_final_df = pd.concat([Company_bill_df, Company_CDS], axis=1)
        Company_final_df.dropna(subset = ['Default'], inplace=True)
        Company_final_df.PD.fillna(1, inplace=True)
        output_path = rf"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Model Input Trial\{Ticker}_final.xlsx"
        Company_final_df.to_excel(output_path)
        print(rf'[{datetime.now()}]{Ticker} is transformed')
    except:
        print(rf'[{datetime.now()}]Failed to transform {Ticker}')
        Error_tickers.append(Ticker)
        continue
# List out company that failed to be transformed
print("Failed to Transform:")
print(Error_tickers)

[2020-12-28 18:46:11.172335]Converting SCMNVX...
[2020-12-28 18:47:28.803673]SCMNVX is transformed
[2020-12-28 18:47:28.804672]Converting SMIN...
[2020-12-28 18:48:34.365452]SMIN is transformed
[2020-12-28 18:48:34.366450]Converting ELISSA...
[2020-12-28 18:50:00.156955]ELISSA is transformed
[2020-12-28 18:50:00.156955]Converting SEE...
[2020-12-28 18:51:06.321584]SEE is transformed
[2020-12-28 18:51:06.321584]Converting A...
[2020-12-28 18:52:10.891112]A is transformed
[2020-12-28 18:52:10.891112]Converting AXP...
[2020-12-28 18:53:17.930558]AXP is transformed
[2020-12-28 18:53:17.930558]Converting BEAER...
[2020-12-28 18:54:25.419439]BEAER is transformed
[2020-12-28 18:54:25.420437]Converting METROB...
[2020-12-28 18:55:31.648008]METROB is transformed
[2020-12-28 18:55:31.648008]Converting SOJICOR...
[2020-12-28 18:56:39.258519]SOJICOR is transformed
[2020-12-28 18:56:39.259515]Converting R...
[2020-12-28 18:57:53.404518]R is transformed
[2020-12-28 18:57:53.405516]Converting NST-EC.

[2020-12-28 20:21:22.367318]NOMURA is transformed
[2020-12-28 20:21:22.368316]Converting GLOPM...
[2020-12-28 20:22:25.774395]GLOPM is transformed
[2020-12-28 20:22:25.774395]Converting BIOGINC...
[2020-12-28 20:23:32.281154]BIOGINC is transformed
[2020-12-28 20:23:32.281154]Converting NESTLE...
[2020-12-28 20:24:37.279476]NESTLE is transformed
[2020-12-28 20:24:37.279476]Converting TEVA...
[2020-12-28 20:25:46.484006]TEVA is transformed
[2020-12-28 20:25:46.485003]Converting COOPER...
[2020-12-28 20:26:51.167651]COOPER is transformed
[2020-12-28 20:26:51.168649]Converting RNTKIL...
[2020-12-28 20:28:01.282769]RNTKIL is transformed
[2020-12-28 20:28:01.282769]Converting MWD...
[2020-12-28 20:29:05.590474]MWD is transformed
[2020-12-28 20:29:05.591473]Converting SABIC...
[2020-12-28 20:30:12.133191]SABIC is transformed
[2020-12-28 20:30:12.135185]Converting LUV...
[2020-12-28 20:31:16.267359]LUV is transformed
[2020-12-28 20:31:16.268358]Converting THCP...
[2020-12-28 20:32:19.923809]TH

[2020-12-28 21:54:04.562823]PEMEX is transformed
[2020-12-28 21:54:04.563822]Converting JWN...
[2020-12-28 21:55:11.454781]JWN is transformed
[2020-12-28 21:55:11.455780]Converting NOR...
[2020-12-28 21:55:26.505597]Failed to transform NOR
[2020-12-28 21:55:26.506595]Converting OI...
[2020-12-28 21:56:33.112335]OI is transformed
[2020-12-28 21:56:33.112335]Converting TOYOBO...
[2020-12-28 21:57:44.682492]TOYOBO is transformed
[2020-12-28 21:57:44.683489]Converting BRTH...
[2020-12-28 21:58:50.509364]BRTH is transformed
[2020-12-28 21:58:50.509364]Converting BMS...
[2020-12-28 21:59:56.930150]BMS is transformed
[2020-12-28 21:59:56.931147]Converting KEIOC...
[2020-12-28 22:01:26.358937]KEIOC is transformed
[2020-12-28 22:01:26.358937]Converting BA...
[2020-12-28 22:02:50.945239]BA is transformed
[2020-12-28 22:02:50.945239]Converting LGUPLS...
[2020-12-28 22:04:02.764404]LGUPLS is transformed
[2020-12-28 22:04:02.765402]Converting TOTO...
[2020-12-28 22:05:10.885970]TOTO is transformed


[2020-12-28 23:25:58.532955]WOR is transformed
[2020-12-28 23:25:58.532955]Converting YAGEO...
[2020-12-28 23:26:13.240558]Failed to transform YAGEO
[2020-12-28 23:26:13.241556]Converting NHFOO...
[2020-12-28 23:27:23.377085]NHFOO is transformed
[2020-12-28 23:27:23.378081]Converting PCU...
[2020-12-28 23:28:27.661963]PCU is transformed
[2020-12-28 23:28:27.662961]Converting NISHIM...
[2020-12-28 23:29:32.726690]NISHIM is transformed
[2020-12-28 23:29:32.727689]Converting FME-Hldg...
[2020-12-28 23:29:47.345538]Failed to transform FME-Hldg
[2020-12-28 23:29:47.345538]Converting EONSE...
[2020-12-28 23:30:52.537936]EONSE is transformed
[2020-12-28 23:30:52.538936]Converting NOKIA...
[2020-12-28 23:31:57.269571]NOKIA is transformed
[2020-12-28 23:31:57.269571]Converting MAR...
[2020-12-28 23:33:02.848934]MAR is transformed
[2020-12-28 23:33:02.849931]Converting CUM...
[2020-12-28 23:34:15.555220]CUM is transformed
[2020-12-28 23:34:15.555220]Converting UVV...
[2020-12-28 23:35:24.290755]

[2020-12-29 00:55:22.918244]TAP is transformed
[2020-12-29 00:55:22.918244]Converting TISINC...
[2020-12-29 00:56:26.542257]TISINC is transformed
[2020-12-29 00:56:26.543255]Converting YARA...
[2020-12-29 00:57:36.687759]YARA is transformed
[2020-12-29 00:57:36.687759]Converting NOV...
[2020-12-29 00:58:40.939100]NOV is transformed
[2020-12-29 00:58:40.939100]Converting FUJIH...
[2020-12-29 00:59:46.256575]FUJIH is transformed
[2020-12-29 00:59:46.257573]Converting FREEPIN...
[2020-12-29 01:00:50.677407]FREEPIN is transformed
[2020-12-29 01:00:50.677407]Converting WLKCC...
[2020-12-29 01:01:56.747970]WLKCC is transformed
[2020-12-29 01:01:56.748969]Converting CANNY...
[2020-12-29 01:03:00.866535]CANNY is transformed
[2020-12-29 01:03:00.867532]Converting METSBOA...
[2020-12-29 01:04:06.655856]METSBOA is transformed
[2020-12-29 01:04:06.655856]Converting NTNCRP...
[2020-12-29 01:05:11.483757]NTNCRP is transformed
[2020-12-29 01:05:11.483757]Converting DDI...
[2020-12-29 01:06:14.784825]

[2020-12-29 02:27:44.936568]TJX is transformed
[2020-12-29 02:27:44.936568]Converting VOEL...
[2020-12-29 02:28:48.270466]VOEL is transformed
[2020-12-29 02:28:48.270466]Converting CASIO...
[2020-12-29 02:29:53.159162]CASIO is transformed
[2020-12-29 02:29:53.160161]Converting ABX...
[2020-12-29 02:30:58.128642]ABX is transformed
[2020-12-29 02:30:58.129640]Converting MSCINC...
[2020-12-29 02:32:01.742783]MSCINC is transformed
[2020-12-29 02:32:01.742783]Converting COH...
[2020-12-29 02:33:05.997316]COH is transformed
[2020-12-29 02:33:05.997316]Converting SGCN...
[2020-12-29 02:33:20.432346]Failed to transform SGCN
[2020-12-29 02:33:20.433343]Converting SUMRUB...
[2020-12-29 02:34:30.093282]SUMRUB is transformed
[2020-12-29 02:34:30.094282]Converting QCOM...
[2020-12-29 02:35:36.444206]QCOM is transformed
[2020-12-29 02:35:36.445205]Converting CKHN...
[2020-12-29 02:36:39.434346]CKHN is transformed
[2020-12-29 02:36:39.434346]Converting IEX...
[2020-12-29 02:36:55.020269]Failed to tra

[2020-12-29 03:58:23.032054]OWC is transformed
[2020-12-29 03:58:23.033052]Converting SUMI...
[2020-12-29 03:59:29.284422]SUMI is transformed
[2020-12-29 03:59:29.285421]Converting BNP...
[2020-12-29 04:00:51.635684]BNP is transformed
[2020-12-29 04:00:51.636681]Converting COMPAL...
[2020-12-29 04:02:28.659444]COMPAL is transformed
[2020-12-29 04:02:28.660440]Converting DAIICLI...
[2020-12-29 04:04:06.526183]DAIICLI is transformed
[2020-12-29 04:04:06.527181]Converting SBIHLD...
[2020-12-29 04:05:15.402488]SBIHLD is transformed
[2020-12-29 04:05:15.402488]Converting CAH...
[2020-12-29 04:06:19.048819]CAH is transformed
[2020-12-29 04:06:19.048819]Converting SCHL...
[2020-12-29 04:07:31.231271]SCHL is transformed
[2020-12-29 04:07:31.231271]Converting RND1...
[2020-12-29 04:07:46.372674]Failed to transform RND1
[2020-12-29 04:07:46.373671]Converting ENI...
[2020-12-29 04:08:50.132725]ENI is transformed
[2020-12-29 04:08:50.133724]Converting TCC...
[2020-12-29 04:09:55.105075]TCC is tran

[2020-12-29 05:30:13.416547]LEN is transformed
[2020-12-29 05:30:13.416547]Converting MITSWT...
[2020-12-29 05:31:17.683555]MITSWT is transformed
[2020-12-29 05:31:17.684554]Converting EJRAIL...
[2020-12-29 05:32:21.042010]EJRAIL is transformed
[2020-12-29 05:32:21.043007]Converting BGG...
[2020-12-29 05:33:40.391477]BGG is transformed
[2020-12-29 05:33:40.392514]Converting JAPRAD...
[2020-12-29 05:34:45.626640]JAPRAD is transformed
[2020-12-29 05:34:45.626640]Converting STJUD...
[2020-12-29 05:35:53.567848]STJUD is transformed
[2020-12-29 05:35:53.567848]Converting IBIDEN...
[2020-12-29 05:36:58.657844]IBIDEN is transformed
[2020-12-29 05:36:58.658842]Converting TMS...
[2020-12-29 05:37:16.260486]Failed to transform TMS
[2020-12-29 05:37:16.261481]Converting ACOM...
[2020-12-29 05:38:20.875634]ACOM is transformed
[2020-12-29 05:38:20.876630]Converting JUSCO...
[2020-12-29 05:39:25.252480]JUSCO is transformed
[2020-12-29 05:39:25.253478]Converting BOMBINC...
[2020-12-29 05:40:29.504677

[2020-12-29 07:01:34.189999]PARFIN-SpA is transformed
[2020-12-29 07:01:34.189999]Converting SUMIEL...
[2020-12-29 07:02:40.034954]SUMIEL is transformed
[2020-12-29 07:02:40.035952]Converting BXP...
[2020-12-29 07:03:44.601366]BXP is transformed
[2020-12-29 07:03:44.602364]Converting MO-PHMI...
[2020-12-29 07:04:50.409422]MO-PHMI is transformed
[2020-12-29 07:04:50.409422]Converting ADM...
[2020-12-29 07:05:56.785086]ADM is transformed
[2020-12-29 07:05:56.785086]Converting ADSYI...
[2020-12-29 07:07:03.017086]ADSYI is transformed
[2020-12-29 07:07:03.017086]Converting PPG...
[2020-12-29 07:08:09.729838]PPG is transformed
[2020-12-29 07:08:09.730879]Converting TORAY...
[2020-12-29 07:09:13.817724]TORAY is transformed
[2020-12-29 07:09:13.817724]Converting MOTSOL...
[2020-12-29 07:10:18.669482]MOTSOL is transformed
[2020-12-29 07:10:18.670520]Converting HON...
[2020-12-29 07:11:23.849496]HON is transformed
[2020-12-29 07:11:23.850493]Converting NIKON...
[2020-12-29 07:12:27.555489]NIKON

[2020-12-29 08:31:40.246501]SKH is transformed
[2020-12-29 08:31:40.246501]Converting CSCO...
[2020-12-29 08:32:48.767677]CSCO is transformed
[2020-12-29 08:32:48.768675]Converting MDC...
[2020-12-29 08:33:53.526944]MDC is transformed
[2020-12-29 08:33:53.527942]Converting NYT...
[2020-12-29 08:34:58.782851]NYT is transformed
[2020-12-29 08:34:58.783848]Converting IFSIA...
[2020-12-29 08:35:15.570816]Failed to transform IFSIA
[2020-12-29 08:35:15.570816]Converting NATUENE...
[2020-12-29 08:36:22.975023]NATUENE is transformed
[2020-12-29 08:36:22.975023]Converting OCR...
[2020-12-29 08:37:32.147963]OCR is transformed
[2020-12-29 08:37:32.148959]Converting APC...
[2020-12-29 08:38:36.139118]APC is transformed
[2020-12-29 08:38:36.140115]Converting NANYA-TECH...
[2020-12-29 08:38:51.365278]Failed to transform NANYA-TECH
[2020-12-29 08:38:51.365278]Converting DOV...
[2020-12-29 08:39:56.519518]DOV is transformed
[2020-12-29 08:39:56.520516]Converting CR...
[2020-12-29 08:41:02.218326]CR is

In [ ]:
# Test Bench for BoL: Testing only and can be ignored
NINEWES_df = pd.read_excel(r"C:\Users\Thomas TH Chow\Desktop\DataLake Team Email 12112020\Company Going to default\NINEWES_bill.xlsx")
NINEWES_df = Transform_BoL(NINEWES_df)
NINEWES_df.to_excel('try.xlsx')

In [ ]:
# Test Bench for CDS: Testing only and can be ignored
input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Import Data\CDS"
cds_dict= {}
for i in range (0,len(cds_files)):
    cds_file = cds_files[i]
    path = os.path.join(input_path, cds_file)
    temp_df = pd.read_parquet(path, columns = ['Tenor','PrimaryCoupon','PrimaryCurve','PD','Year','Month','Ticker'])
    temp_df = temp_df[temp_df['Ticker'] == Ticker]
    if len(temp_df) !=0:
        temp_dict = temp_df.to_dict('index')
        cds_dict.update(temp_dict)
    index = temp_df.index
del temp_df
del temp_dict
gc.collect()
Company_CDS = pd.DataFrame.from_dict(cds_dict, orient='index')
Company_CDS = Transform_CDS(Company_CDS, Ticker)
Company_CDS.to_excel('try3.xlsx')

In [ ]:
# Test Bench for CDS: Testing only and can be ignored
Ticker = 'NINEWES'
NINEWES_CDS = pd.read_excel(r"C:\Users\Thomas TH Chow\Desktop\DataLake Team Email 12112020\Company Going to default\NINEWES.xlsx")
NINEWES_CDS = Transform_CDS(NINEWES_CDS, Ticker)
NINEWES_CDS.to_excel('try1.xlsx')


In [ ]:
# Test Bench for BoL: Testing only and can be ignored
NINEWES_df = pd.read_excel(r"C:\Users\Thomas TH Chow\Desktop\DataLake Team Email 12112020\Company Going to default\NINEWES_bill.xlsx")
NINEWES_df = Transform_BoL(NINEWES_df)
#Country_df = Count_country(NINEWES_df, 'foreign_company_country')
NINEWES_df.to_excel('try.xlsx')

In [4]:
# Optional codes for visualisation of complete BoL and CDS records of a specific company, e.g. NINEWES, JCL and A
# Convert them into separate Excels to locate potential features
Tickers = ['A']

for Ticker in Tickers:
    input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\CDS"
    cds_files = [file for file in listdir(input_path) if file.endswith('.parquet.gzip')]
    for i in range (0,len(cds_files)):
        cds_file = cds_files[i]
        path = os.path.join(input_path, cds_file)
        temp_df = pd.read_parquet(path)
        temp_df = temp_df[temp_df['Ticker'] == Ticker]
        temp_df.shape
        if i == 0:
            default_df = temp_df
        else:
            default_df = default_df.append(temp_df, sort=False)
        default_df.shape
        del temp_df
        gc.collect()
    default_df.shape
    default_df.to_excel(f'{Ticker}.xlsx')

    input_path = r"C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading"
    Bill_Temps = [file for file in listdir(input_path) if file.endswith('.parquet.gzip')]
    for j in range (0,len(Bill_Temps)):
        Bill_Temp = Bill_Temps[j]
        path = os.path.join(input_path, Bill_Temp)
        Temp_df = pd.read_parquet(path)
        Temp_df = Temp_df[Temp_df['matched_fcompany_ihsm_ticker'] == Ticker]
        if (j==0):
            default_bill_df = Temp_df
        else: 
            default_bill_df = default_bill_df.append(Temp_df, sort=False)
        del Temp_df
        gc.collect()
    default_bill_df.shape
    default_bill_df.to_excel(f'{Ticker}_bill.xlsx')

C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading\Bill_export Part1.parquet.gzip
(71222, 16)
C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading\Bill_export Part10.parquet.gzip
(70625, 16)
C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading\Bill_export Part11.parquet.gzip
(89146, 16)
C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading\Bill_export Part12.parquet.gzip
(70731, 16)
C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading\Bill_export Part13.parquet.gzip
(71009, 16)
C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of Lading\Bill_export Part14.parquet.gzip
(71616, 16)
C:\Users\Thomas TH Chow\Desktop\Datalake\Credit Rating Modeling\Clean datasets\Export Data\Bill of La